# DEM Retriever

This notebook is a secondary notebook, supposed to get the digital elevation model from opentopography.org

## Getting DEM-Data from [opentopography](https://portal.opentopography.org/apidocs/#/Public/getGlobalDem)
To run this notebook you will need a (free) API key from *opentopography.org*. 
I’ve got myself one and put it into the environment variables.

In [1]:
# All the imports

# For getting the DEM-Data (from opentopography.org) 
import requests

# Handling the API-Key (hidden from this repo)
from dotenv import load_dotenv

# Filesystem, JSON
import os
import json
from io import BytesIO # To be able to route the API response to a file-like object that rasterio can use

# Rasterio for handling GeoTIFFs
import rasterio

# Math may not be missing
import math


In [2]:
# Let this notebook load the API key from the .env file
load_dotenv()

# Create a dictionary for settings that are not saved
external_settings = {}
external_settings["opentopo_api_key"] = os.getenv("OPENTOPO_API_KEY")

### Hint: API Key
If you want to use your own API-Key, you’ll have to assign it in the following code block. Remember: You will have to get your own API key from opentopography.org to make this work.

In [3]:

# Hint! If you want to use your own API-Key, you’ll have to assign it here:
# external_settings["opentopo_api_key"]

print(external_settings["opentopo_api_key"])

352c27c644b3a6e0132002d187f75a57


## Getting the DEM-Data
For this example we’ll use the EU_DTM dataset which is very accurate for europe and resembles bare earth heights.

In [ ]:
# Create a variable for settings that can later be saved as a JSON-file
internal_settings = {}

internal_settings["geo"] = {}
internal_settings["geo"]["demtype"] = "COP90"
internal_settings["geo"]["south"] = 46.93
internal_settings["geo"]["north"] = 48.02
internal_settings["geo"]["west"] = 6.5
internal_settings["geo"]["east"] = 8.6
internal_settings["geo"]["outputFormat"] = "GTiff"

# When the average difference of the given bounds to those of an existing geotiff image 
# is less than this threshold, the GeoTIFF is considered already existent. 
# This is to prevent another api call 
internal_settings["geo"]["tolerance"] = 0.5 # In degrees. Could be set lower if the regions become very small

internal_settings["files"] = {}
internal_settings["files"]["intermediary_folder"] = "intermediary"

# Important: Geotiffs have to have the extension "TIF" to be readable by google earth pro...x
internal_settings["files"]["geotiff"] = "geotiff.tif" 


# Generate temporary settings
temporary_settings = {}
temporary_settings["dem_file_location"] = os.path.join(internal_settings["files"]
                                                       ["intermediary_folder"],
                                                       internal_settings["files"]["geotiff"])

temporary_settings["dem_already_existent"] = False


In [ ]:
# Before calling the api again, we should check if there’s a file existing that is exactly like the query 
if os.path.exists(temporary_settings["dem_file_location"]):
    with rasterio.open(temporary_settings["dem_file_location"]) as src:
        average_error = (abs(src.bounds.left - internal_settings["geo"]["west"]) + abs(src.bounds.right - internal_settings["geo"]["east"]) + abs(src.bounds.top - internal_settings["geo"]["north"]) + abs(src.bounds.bottom - internal_settings["geo"]["south"])) / 4

        if average_error < internal_settings["geo"]["tolerance"]: temporary_settings["dem_already_existent"] = True

In [6]:
# Get the API key into the request data
api_call_data = json.loads(json.dumps(internal_settings["geo"]))
api_call_data.update({"API_Key": external_settings["opentopo_api_key"]})


In [ ]:
if temporary_settings["dem_already_existent"]:
        print("Comparable DEM file already exists, skipping API call.")
else:
    response = requests.get("https://portal.opentopography.org/API/globaldem", params=api_call_data)
    with open(temporary_settings["dem_file_location"], "wb") as file:
        file.write(response.content)


Comparable DEM file already exists, skipping API call.


# Aufteilen
Die DEM Daten in Kacheln aufteilen, die 100 km groß sind